In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# github_recsys_twotower_with_robust_eval.py

import os
import math
import random
from typing import Dict

import numpy as np
import pandas as pd
import torch
from torch import nn
from torch.utils.data import Dataset, DataLoader

# Optional: for AUC metric
try:
    from sklearn.metrics import roc_auc_score
    SKLEARN_AVAILABLE = True
except ImportError:
    SKLEARN_AVAILABLE = False


########################################
# 1. CONFIG AND FEATURE DEFINITIONS
########################################

class Config:
    # File paths: adapt these to your actual locations
    train_balanced_path = "/content/drive/MyDrive/Project_Work/RepoRecSys/data/train_balanced.csv"
    train_negative_path = "/content/drive/MyDrive/Project_Work/RepoRecSys/data/train_negative.csv"
    test_balanced_path = "/content/drive/MyDrive/Project_Work/RepoRecSys/data/test_balanced.csv"
    test_negative_path = "/content/drive/MyDrive/Project_Work/RepoRecSys/data/test_negative.csv"

    # Model hyperparameters
    user_id_emb_dim = 64
    item_id_emb_dim = 64
    lang_emb_dim = 16
    hidden_dim = 128
    embedding_dim = 64  # final tower output dimension

    dropout = 0.0

    batch_size = 4096
    num_epochs = 2
    lr = 1e-3
    weight_decay = 1e-5

    # Contrastive Loss Config
    temperature = 0.1  # Scale factor for cosine similarity logits

    # Validation split
    train_user_fraction = 0.8  # 80% users for train, 20% for validation

    # Ranking evaluation config
    eval_k_list = [5, 10]
    eval_num_negatives = 100  # per user for ranking eval

    # CUDA if available
    device = "cuda" if torch.cuda.is_available() else "cpu"

    # Random seeds for reproducibility
    seed = 42

    # Debug options
    debug_small_train = False   # set True to overfit tiny subset
    debug_small_train_size = 2000  # number of positive rows to keep

    # === README text embedding config (Stage 4) ===
    use_readme_text = True  # toggle this to turn text features on/off

    # Paths to the outputs from build_readme_embeddings.py (Stage 4)
    readme_emb_npy_path = "/content/drive/MyDrive/Project_Work/RepoRecSys/data/features/project_readme_embeddings.npy"
    readme_emb_index_path = "/content/drive/MyDrive/Project_Work/RepoRecSys/data/features/project_readme_embeddings_index.csv"

    # How to use the text embeddings inside the item tower
    readme_text_hidden_dim = 64   # projection size for text branch
    freeze_text_emb = True        # set False if you want to fine-tune them

# Columns in csvs
NUMERIC_REPO_COLS = [
    "watchers", "commits", "issues", "pull_requests",
    "mean_commits_language", "max_commits_language", "min_commits_language",
    "std_commits_language",
    "mean_pull_requests_language", "max_pull_requests_language",
    "min_pull_requests_language", "std_pull_requests_language",
    "mean_issues_language", "max_issues_language", "min_issues_language",
    "std_issues_language",
    "mean_watchers_language", "max_watchers_language",
    "min_watchers_language", "std_watchers_language",
    "events", "year",
    "weight", "cp", "avg_cp", "stddev",
]

CATEGORICAL_REPO_COLS = [
    "language_code",
]

USER_ID_COL = "id_user"
ITEM_ID_COL = "project_id"
TARGET_COL = "target"

In [ ]:
########################################
# 2. DATA LOADING AND PREPROCESSING
########################################

def set_seed(seed: int):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)


def load_raw_data(cfg: Config):
    train_bal = pd.read_csv(cfg.train_balanced_path)
    train_neg = pd.read_csv(cfg.train_negative_path)
    test_bal = pd.read_csv(cfg.test_balanced_path)
    test_neg = pd.read_csv(cfg.test_negative_path)
    return train_bal, train_neg, test_bal, test_neg


def build_id_mappings(train_pool: pd.DataFrame, test_df: pd.DataFrame):
    # Unique users and items across train_pool + test
    all_users = pd.concat([train_pool[USER_ID_COL], test_df[USER_ID_COL]]).unique()
    all_items = pd.concat([train_pool[ITEM_ID_COL], test_df[ITEM_ID_COL]]).unique()
    all_langs = pd.concat([train_pool["language_code"], test_df["language_code"]]).unique()

    user2idx = {uid: i for i, uid in enumerate(all_users)}
    item2idx = {iid: i for i, iid in enumerate(all_items)}
    lang2idx = {lid: i for i, lid in enumerate(all_langs)}

    idx2user = {i: uid for uid, i in user2idx.items()}
    idx2item = {i: iid for iid, i in item2idx.items()}
    idx2lang = {i: lid for lid, i in lang2idx.items()}

    return user2idx, item2idx, lang2idx, idx2user, idx2item, idx2lang


def compute_numeric_scalers(train_pool: pd.DataFrame):
    """
    Compute mean and std for each numeric repo feature on training pool data.
    Returns dicts: col -> (mean, std).
    """
    means = {}
    stds = {}
    for col in NUMERIC_REPO_COLS:
        col_values = train_pool[col].astype(float).values
        mean = col_values.mean()
        std = col_values.std()
        if std < 1e-6:
            std = 1.0
        means[col] = mean
        stds[col] = std
    return means, stds


def normalize_numeric_features(df: pd.DataFrame, means: Dict[str, float], stds: Dict[str, float]):
    df = df.copy()
    for col in NUMERIC_REPO_COLS:
        df[col] = (df[col].astype(float) - means[col]) / stds[col]
    return df


def split_train_val_by_user(
    train_pool: pd.DataFrame,
    train_user_fraction: float,
    seed: int,
):
    """
    Split the training pool into train_df and val_df based on users.
    """
    all_users = train_pool[USER_ID_COL].unique()
    rng = np.random.RandomState(seed)
    rng.shuffle(all_users)

    n_train_users = int(len(all_users) * train_user_fraction)
    train_users = set(all_users[:n_train_users])
    val_users = set(all_users[n_train_users:])

    train_df = train_pool[train_pool[USER_ID_COL].isin(train_users)].reset_index(drop=True)
    val_df = train_pool[train_pool[USER_ID_COL].isin(val_users)].reset_index(drop=True)

    return train_df, val_df


def build_item_feature_table_norm(
    train_pool: pd.DataFrame,
    test_df: pd.DataFrame,
    means: Dict[str, float],
    stds: Dict[str, float],
):
    """
    Build a unique, normalized item feature table: one row per project_id.
    """
    all_df = pd.concat([train_pool, test_df], ignore_index=True)
    all_df_norm = normalize_numeric_features(all_df, means, stds)

    # Keep one row per item; using last occurrence (could also aggregate)
    all_df_norm = all_df_norm.sort_values("events")
    item_table = all_df_norm.drop_duplicates(subset=[ITEM_ID_COL], keep="last")
    item_table = item_table.reset_index(drop=True)
    return item_table

def get_readme_project_ids(cfg: Config) -> set[int]:
    """
    Return the set of project_ids that have a README embedding.
    """
    if not cfg.use_readme_text:
        return set()

    idx_df = pd.read_csv(cfg.readme_emb_index_path)
    # make sure we drop any NaNs and cast to int
    pids = idx_df["project_id"].dropna().astype("int64").unique().tolist()
    return set(int(pid) for pid in pids)

import numpy as np

def load_item_text_embeddings(cfg: Config, item2idx: dict) -> tuple[torch.Tensor, int]:
    """
    Build an [num_items, text_dim] matrix where each row is the README embedding
    for that item, aligned with item2idx. Items without README get all zeros.

    Returns:
        item_text_emb_tensor: torch.FloatTensor[num_items, text_dim]
        text_dim: int
    """
    if not cfg.use_readme_text:
        return None, 0

    # Load index and embedding matrix produced in Stage 4
    idx_df = pd.read_csv(cfg.readme_emb_index_path)
    emb = np.load(cfg.readme_emb_npy_path)  # [N_text_items, D]
    text_dim = emb.shape[1]

    # Map project_id -> row_idx into emb
    row_by_pid = {
        int(pid): int(row_idx)
        for pid, row_idx in zip(idx_df["project_id"], idx_df["row_idx"])
    }

    num_items = len(item2idx)
    item_text_emb = np.zeros((num_items, text_dim), dtype=np.float32)

    missing = 0
    for raw_project_id, item_idx in item2idx.items():
        row_idx = row_by_pid.get(int(raw_project_id))
        if row_idx is not None:
            item_text_emb[item_idx] = emb[row_idx]
        else:
            # no README embedding available for this project_id
            missing += 1

    print(f"[Text] Loaded README embeddings for {num_items - missing} / {num_items} items "
          f"({100.0 * (num_items - missing) / num_items:.2f}%).")

    item_text_emb_tensor = torch.from_numpy(item_text_emb)
    return item_text_emb_tensor, text_dim


In [ ]:
########################################
# 3. DATASET AND DATALOADER
########################################

class TwoTowerDataset(Dataset):
    def __init__(
        self,
        df: pd.DataFrame,
        user2idx: Dict[int, int],
        item2idx: Dict[int, int],
        lang2idx: Dict[int, int],
    ):
        self.df = df.reset_index(drop=True)
        self.user2idx = user2idx
        self.item2idx = item2idx
        self.lang2idx = lang2idx

        # Pre-extract numpy arrays so __getitem__ is fast
        self.user_ids = self.df[USER_ID_COL].values
        self.item_ids = self.df[ITEM_ID_COL].values
        self.lang_codes = self.df["language_code"].values
        self.labels = self.df[TARGET_COL].astype(float).values

        base_numeric = self.df[NUMERIC_REPO_COLS].astype(float).values
        has_readme = self.df["has_readme"].astype(float).values.reshape(-1, 1)
        self.numeric_matrix = np.concatenate([base_numeric, has_readme], axis=1)


    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx: int):
        uid = self.user_ids[idx]
        iid = self.item_ids[idx]
        lang = self.lang_codes[idx]
        label = self.labels[idx]
        numerics = self.numeric_matrix[idx]

        u_idx = self.user2idx[uid]
        i_idx = self.item2idx[iid]
        l_idx = self.lang2idx[lang]

        # Convert to tensors
        u_idx = torch.tensor(u_idx, dtype=torch.long)
        i_idx = torch.tensor(i_idx, dtype=torch.long)
        l_idx = torch.tensor(l_idx, dtype=torch.long)
        numerics = torch.tensor(numerics, dtype=torch.float32)
        label = torch.tensor(label, dtype=torch.float32)

        return u_idx, i_idx, l_idx, numerics, label


def make_dataloaders(
    cfg: Config,
    train_df: pd.DataFrame,
    val_df: pd.DataFrame,
    user2idx: Dict[int, int],
    item2idx: Dict[int, int],
    lang2idx: Dict[int, int],
):
    train_dataset = TwoTowerDataset(train_df, user2idx, item2idx, lang2idx)
    val_dataset = TwoTowerDataset(val_df, user2idx, item2idx, lang2idx)

    train_loader = DataLoader(
        train_dataset,
        batch_size=cfg.batch_size,
        shuffle=True,
        num_workers=2,
        pin_memory=True,
    )

    val_loader = DataLoader(
        val_dataset,
        batch_size=cfg.batch_size,
        shuffle=False,
        num_workers=2,
        pin_memory=True,
    )

    return train_loader, val_loader

In [ ]:
########################################
# 4. MODEL DEFINITIONS
########################################

class UserTower(nn.Module):
    """
    User tower with:
      - user ID embedding
      - 2-layer MLP + dropout
      - L2-normalized output
    """
    def __init__(self, num_users: int, cfg: Config):
        super().__init__()
        self.user_emb = nn.Embedding(num_users, cfg.user_id_emb_dim)

        self.mlp = nn.Sequential(
            nn.Linear(cfg.user_id_emb_dim, cfg.hidden_dim),
            nn.ReLU(),
            nn.Dropout(cfg.dropout),
            nn.Linear(cfg.hidden_dim, cfg.hidden_dim),
            nn.ReLU(),
            nn.Dropout(cfg.dropout),
            nn.Linear(cfg.hidden_dim, cfg.embedding_dim),
        )

    def forward(self, user_ids: torch.Tensor) -> torch.Tensor:
        # [B] -> [B, user_id_emb_dim]
        x = self.user_emb(user_ids)

        # [B, user_id_emb_dim] -> [B, embedding_dim]
        x = self.mlp(x)

        # Normalize to keep dot product ≈ cosine similarity
        x = nn.functional.normalize(x, p=2, dim=-1)
        return x



class ItemTower(nn.Module):
    """
    Item tower with:
      - item ID embedding
      - language embedding
      - small MLP on numeric features
      - optional README text embedding branch
    """
    def __init__(
        self,
        num_items: int,
        num_langs: int,
        num_numeric_feats: int,
        text_dim: int,
        item_text_emb_matrix: torch.Tensor | None,
        cfg: Config,
    ):
        super().__init__()

        self.cfg = cfg
        self.use_text = cfg.use_readme_text and (text_dim > 0) and (item_text_emb_matrix is not None)

        # ID + language embeddings (as before)
        self.item_emb = nn.Embedding(num_items, cfg.item_id_emb_dim)
        self.lang_emb = nn.Embedding(num_langs, cfg.lang_emb_dim)

        # Numeric features MLP (Stage 3)
        self.num_mlp = nn.Sequential(
            nn.Linear(num_numeric_feats, 64),
            nn.ReLU(),
            nn.Linear(64, 32),
            nn.ReLU(),
        )

        # Optional text embedding branch
        if self.use_text:
            # Embedding table indexed by item_id (same indices as item_emb)
            # from_pretrained will move weights to correct device automatically
            self.text_emb = nn.Embedding.from_pretrained(
                item_text_emb_matrix.float(), freeze=cfg.freeze_text_emb
            )

            self.text_proj = nn.Sequential(
                nn.Linear(text_dim, cfg.readme_text_hidden_dim),
                nn.ReLU(),
            )
            text_output_dim = cfg.readme_text_hidden_dim
        else:
            self.text_emb = None
            self.text_proj = None
            text_output_dim = 0

        # Fused MLP: concat [item_emb, lang_emb, num_e, text_e]
        fused_input_dim = cfg.item_id_emb_dim + cfg.lang_emb_dim + 32 + text_output_dim

        self.mlp = nn.Sequential(
            nn.Linear(fused_input_dim, cfg.hidden_dim),
            nn.ReLU(),
            nn.Linear(cfg.hidden_dim, cfg.embedding_dim),
        )

    def forward(
        self,
        item_ids: torch.Tensor,
        lang_ids: torch.Tensor,
        numeric_feats: torch.Tensor,
    ) -> torch.Tensor:
        item_e = self.item_emb(item_ids)       # [B, item_id_emb_dim]
        lang_e = self.lang_emb(lang_ids)       # [B, lang_emb_dim]
        num_e = self.num_mlp(numeric_feats)    # [B, 32]

        feats = [item_e, lang_e, num_e]

        if self.use_text:
            # text_emb: [B, text_dim] -> [B, text_hidden_dim]
            text_raw = self.text_emb(item_ids)
            text_e = self.text_proj(text_raw)
            feats.append(text_e)

        x = torch.cat(feats, dim=-1)           # [B, fused_input_dim]
        x = self.mlp(x)                        # [B, embedding_dim]
        x = nn.functional.normalize(x, p=2, dim=-1)
        return x




class TwoTowerRecSys(nn.Module):
    def __init__(
        self,
        num_users: int,
        num_items: int,
        num_langs: int,
        num_numeric_feats: int,
        text_dim: int,
        item_text_emb_matrix: torch.Tensor | None,
        cfg: Config,
    ):
        super().__init__()
        self.user_tower = UserTower(num_users, cfg)
        self.item_tower = ItemTower(
            num_items=num_items,
            num_langs=num_langs,
            num_numeric_feats=num_numeric_feats,
            text_dim=text_dim,
            item_text_emb_matrix=item_text_emb_matrix,
            cfg=cfg,
        )


    def forward(
        self,
        user_ids: torch.Tensor,
        item_ids: torch.Tensor,
        lang_ids: torch.Tensor,
        numeric_feats: torch.Tensor,
    ):
        """
        user_ids: [B]
        item_ids: [B]
        lang_ids: [B]
        numeric_feats: [B, F]

        returns:
            logits: [B] (dot products)
            user_embs: [B, D]
            item_embs: [B, D]
        """
        u = self.user_tower(user_ids)                          # [B, D]
        v = self.item_tower(item_ids, lang_ids, numeric_feats) # [B, D]

        logits = torch.sum(u * v, dim=-1)  # [B]
        return logits, u, v

In [ ]:
def train_one_epoch(
    model: TwoTowerRecSys,
    train_loader: DataLoader,
    optimizer: torch.optim.Optimizer,
    device: str,
    temperature: float = 0.1,
):
    model.train()
    total_loss = 0.0
    total_examples = 0

    # Use CrossEntropyLoss for In-Batch Negatives (Contrastive Learning)
    criterion = nn.CrossEntropyLoss()

    for batch in train_loader:
        # Unpack batch
        # Note: In this mode, we expect 'batch' to contain only POSITIVE pairs.
        # 'labels' from the dataset are ignored because the targets are implicit (the diagonal).
        user_ids, item_ids, lang_ids, numerics, _ = batch

        user_ids = user_ids.to(device)
        item_ids = item_ids.to(device)
        lang_ids = lang_ids.to(device)
        numerics = numerics.to(device)

        optimizer.zero_grad()

        # Get embeddings from the model
        # forward() returns: logits, user_embs, item_embs
        # We ignore the pointwise 'logits' and use the embeddings directly.
        _, u_emb, v_emb = model(user_ids, item_ids, lang_ids, numerics)

        # Compute Similarity Matrix: [Batch_Size, Batch_Size]
        # Both u_emb and v_emb are L2 normalized in the model, so this is Cosine Similarity.
        sim_matrix = torch.matmul(u_emb, v_emb.T)

        # Scale by temperature to sharpen the distribution
        logits = sim_matrix / temperature

        # The target for user i is item i (the diagonal)
        batch_size = user_ids.size(0)
        targets = torch.arange(batch_size, device=device, dtype=torch.long)

        loss = criterion(logits, targets)
        loss.backward()
        optimizer.step()

        total_loss += loss.item() * batch_size
        total_examples += batch_size

    avg_loss = total_loss / max(total_examples, 1)
    return avg_loss


def evaluate_pointwise(
    model: TwoTowerRecSys,
    data_loader: DataLoader,
    device: str,
):
    """
    Simple pointwise evaluation:
    - average loss
    - ROC-AUC (if sklearn available)
    """
    model.eval()
    criterion = nn.BCEWithLogitsLoss()

    total_loss = 0.0
    total_examples = 0

    all_labels = []
    all_probs = []

    with torch.no_grad():
        for batch in data_loader:
            user_ids, item_ids, lang_ids, numerics, labels = batch
            user_ids = user_ids.to(device)
            item_ids = item_ids.to(device)
            lang_ids = lang_ids.to(device)
            numerics = numerics.to(device)
            labels = labels.to(device)

            logits, _, _ = model(user_ids, item_ids, lang_ids, numerics)
            loss = criterion(logits, labels)

            probs = torch.sigmoid(logits)

            batch_size = labels.size(0)
            total_loss += loss.item() * batch_size
            total_examples += batch_size

            all_labels.append(labels.detach().cpu().numpy())
            all_probs.append(probs.detach().cpu().numpy())

    avg_loss = total_loss / max(total_examples, 1)
    all_labels = np.concatenate(all_labels, axis=0)
    all_probs = np.concatenate(all_probs, axis=0)

    if SKLEARN_AVAILABLE:
        try:
            auc = roc_auc_score(all_labels, all_probs)
        except ValueError:
            auc = float("nan")
    else:
        auc = float("nan")

    return avg_loss, auc

In [ ]:
########################################
# 6. ROBUST RANKING METRICS (Recall@K, NDCG@K)
########################################

def compute_dcg(relevances: np.ndarray, k: int) -> float:
    """
    relevances: binary or graded relevance scores sorted in ranking order.
    """
    rel = relevances[:k]
    if len(rel) == 0:
        return 0.0
    discounts = np.log2(np.arange(2, len(rel) + 2))
    return float(np.sum((2**rel - 1) / discounts))


def evaluate_ranking_with_neg_sampling(
    model: TwoTowerRecSys,
    test_df: pd.DataFrame,
    train_pool: pd.DataFrame,
    item_table: pd.DataFrame,
    user2idx: Dict[int, int],
    item2idx: Dict[int, int],
    lang2idx: Dict[int, int],
    device: str,
    k_list=None,
    num_negatives: int = 100,
):
    """
    More robust ranking evaluation:
    For each user in test_df:
      - Positives = repos in test_df with target=1.
      - Negatives = sample 'num_negatives' items from full catalog that the user
        has never positively interacted with (train_pool + test_df).
      - Rank positive + sampled negatives with the model.
      - Compute Recall@K and NDCG@K for each K.

    Returns: dict like
      {
        "Recall@5": value,
        "Recall@10": value,
        "NDCG@5": value,
        "NDCG@10": value,
        "num_users": n_users_evaluated
      }
    """
    if k_list is None:
        k_list = [5, 10]

    model.eval()

    # Pre-build mapping from project_id -> row index in item_table
    item_row_map = {pid: idx for idx, pid in enumerate(item_table[ITEM_ID_COL].values)}
    all_item_ids = item_table[ITEM_ID_COL].values

    # For each user: set of items they have positively interacted with anywhere
    user_pos_all = (
        pd.concat([train_pool, test_df], ignore_index=True)
        .query(f"{TARGET_COL} == 1")
        .groupby(USER_ID_COL)[ITEM_ID_COL]
        .agg(lambda x: set(x.values))
    )

    # Positives only from test
    user_pos_test = (
        test_df.query(f"{TARGET_COL} == 1")
        .groupby(USER_ID_COL)[ITEM_ID_COL]
        .agg(lambda x: list(set(x.values)))
    )

    # Users we can evaluate on (have positives in test and exist in mapping)
    eval_users = [
        uid for uid in user_pos_test.index
        if (uid in user2idx) and (uid in user_pos_all.index)
    ]

    recalls = {k: [] for k in k_list}
    ndcgs = {k: [] for k in k_list}

    with torch.no_grad():
        for uid in eval_users:
            pos_items = user_pos_test[uid]
            if len(pos_items) == 0:
                continue

            all_pos = user_pos_all[uid]
            seen_items = set(all_pos)

            # Candidate negative pool = all items - seen positives
            candidate_neg = [pid for pid in all_item_ids if pid not in seen_items]
            if len(candidate_neg) == 0:
                continue

            # Sample negatives
            num_neg = min(num_negatives, len(candidate_neg))
            neg_items = random.sample(candidate_neg, num_neg)

            # Build candidate list: positives + negatives
            cand_items = list(pos_items) + neg_items
            cand_labels = np.array(
                [1] * len(pos_items) + [0] * len(neg_items),
                dtype=np.int32,
            )

            # Map to item_table rows
            cand_indices = [item_row_map[pid] for pid in cand_items]
            cand_rows = item_table.iloc[cand_indices]

            item_ids_raw = cand_rows[ITEM_ID_COL].values
            lang_raw = cand_rows["language_code"].values

            item_indices = torch.tensor(
                [item2idx[i] for i in item_ids_raw],
                dtype=torch.long,
                device=device,
            )
            lang_indices = torch.tensor(
                [lang2idx[l] for l in lang_raw],
                dtype=torch.long,
                device=device,
            )
            base_numeric = cand_rows[NUMERIC_REPO_COLS].astype(float).values
            has_readme = cand_rows["has_readme"].astype(float).values.reshape(-1, 1)
            numerics = np.concatenate([base_numeric, has_readme], axis=1)
            numerics_tensor = torch.tensor(
                numerics,
                dtype=torch.float32,
                device=device,
            )

            u_idx = user2idx[uid]
            user_tensor = torch.tensor([u_idx], dtype=torch.long, device=device)
            user_batch = user_tensor.expand(len(cand_items))

            logits, _, _ = model(user_batch, item_indices, lang_indices, numerics_tensor)
            scores = torch.sigmoid(logits).detach().cpu().numpy()

            # Sort candidates by score
            ranking = np.argsort(-scores)  # descending
            sorted_labels = cand_labels[ranking]

            num_pos = float(len(pos_items))

            for k in k_list:
                k_eff = min(k, len(sorted_labels))
                topk_labels = sorted_labels[:k_eff]

                # Recall@K: how many positives in top-K divided by total positives
                recall_k = float(topk_labels.sum() / num_pos)
                recalls[k].append(recall_k)

                # NDCG@K
                dcg_k = compute_dcg(sorted_labels, k_eff)
                # Ideal DCG: all positives ranked at top
                ideal_labels = np.sort(cand_labels)[::-1]
                idcg_k = compute_dcg(ideal_labels, k_eff)
                ndcg_k = dcg_k / idcg_k if idcg_k > 0 else 0.0
                ndcgs[k].append(ndcg_k)

    metrics = {}
    n_users = len(eval_users)
    metrics["num_users"] = n_users
    for k in k_list:
        if len(recalls[k]) > 0:
            metrics[f"Recall@{k}"] = float(np.mean(recalls[k]))
        else:
            metrics[f"Recall@{k}"] = float("nan")
        if len(ndcgs[k]) > 0:
            metrics[f"NDCG@{k}"] = float(np.mean(ndcgs[k]))
        else:
            metrics[f"NDCG@{k}"] = float("nan")

    return metrics

In [ ]:
########################################
# 7. MAIN SCRIPT
########################################

def main():
    cfg = Config()
    set_seed(cfg.seed)

    print("Loading raw data...")
    train_bal, train_neg, test_bal, test_neg = load_raw_data(cfg)

    train_pool = pd.concat([train_bal, train_neg], ignore_index=True)
    test_df = pd.concat([test_bal, test_neg], ignore_index=True)

    print(f"Train pool size: {len(train_pool)}, Test size: {len(test_df)}")

    print("Splitting train pool into train and validation by user...")
    train_df_all, val_df = split_train_val_by_user(
        train_pool,
        train_user_fraction=cfg.train_user_fraction,
        seed=cfg.seed,
    )

    # IMPORTANT for In-Batch Negatives:
    # We only train on POSITIVE interactions. The "negatives" are implicitly
    # the other items in the batch.
    print("Filtering training set to keep only POSITIVES (target=1)...")
    train_df = train_df_all[train_df_all[TARGET_COL] == 1].reset_index(drop=True)

    # We keep val_df as is (mixed pos/neg) so we can still calculate pointwise AUC
    # as a sanity check.

    print(f"Train size (pos only): {len(train_df)}, Val size (mixed): {len(val_df)}")

    print("Building ID mappings (users/items/langs)...")
    user2idx, item2idx, lang2idx, idx2user, idx2item, idx2lang = build_id_mappings(
        train_pool, test_df
    )
    num_users = len(user2idx)
    num_items = len(item2idx)
    num_langs = len(lang2idx)

    print(f"Num users: {num_users}, num items: {num_items}, num langs: {num_langs}")

    num_numeric_feats = len(NUMERIC_REPO_COLS) + 1

    # Load item text embeddings (can be None if disabled)
    item_text_emb_matrix, text_dim = load_item_text_embeddings(cfg, item2idx)
    if not cfg.use_readme_text:
        text_dim = 0  # make sure model knows there is no text branch

    if cfg.debug_small_train:
        # Overfit on a tiny subset of positives
        train_df = train_df.sample(
            n=min(cfg.debug_small_train_size, len(train_df)),
            random_state=cfg.seed,
        ).reset_index(drop=True)
        print(f"[DEBUG] Using small training subset: {len(train_df)} rows (positives only)")

    # Load the set of project_ids that have README embeddings
    readme_project_ids = get_readme_project_ids(cfg)

    def add_has_readme_flag(df: pd.DataFrame, readme_ids: set[int]) -> pd.DataFrame:
        # boolean mask -> float32 (0.0 or 1.0)
        df["has_readme"] = df[ITEM_ID_COL].astype("int64").isin(readme_ids).astype("float32")
        return df

    train_pool = add_has_readme_flag(train_pool, readme_project_ids)
    train_df   = add_has_readme_flag(train_df,   readme_project_ids)
    val_df     = add_has_readme_flag(val_df,     readme_project_ids)
    test_df    = add_has_readme_flag(test_df,    readme_project_ids)

    print("Fitting numeric scalers on training pool...")
    means, stds = compute_numeric_scalers(train_pool)

    print("Normalizing numeric features...")
    train_df_norm = normalize_numeric_features(train_df, means, stds)
    val_df_norm = normalize_numeric_features(val_df, means, stds)
    test_df_norm = normalize_numeric_features(test_df, means, stds)

    print("Creating dataloaders for train and validation...")
    train_loader, val_loader = make_dataloaders(
        cfg, train_df_norm, val_df_norm, user2idx, item2idx, lang2idx
    )

    print("Initializing model...")
    model = TwoTowerRecSys(
    num_users=num_users,
    num_items=num_items,
    num_langs=num_langs,
    num_numeric_feats=num_numeric_feats,
    text_dim=text_dim,
    item_text_emb_matrix=item_text_emb_matrix,
    cfg=cfg,
    ).to(cfg.device)

    optimizer = torch.optim.Adam(
        model.parameters(),
        lr=cfg.lr,
        weight_decay=cfg.weight_decay,
    )

    best_auc = -1.0

    for epoch in range(1, cfg.num_epochs + 1):
        print(f"\nEpoch {epoch}/{cfg.num_epochs}")

        # Pass temperature to train function
        train_loss = train_one_epoch(model, train_loader, optimizer, cfg.device, cfg.temperature)
        print(f"Train loss (CrossEntropy): {train_loss:.4f}")

        val_loss, val_auc = evaluate_pointwise(model, val_loader, cfg.device)
        print(f"Val loss (Pointwise): {val_loss:.4f}, Val AUC: {val_auc:.4f}")

        # Note: best model selection is still based on AUC here, which is fine,
        # but ideally we would track Recall@K on validation.
        if SKLEARN_AVAILABLE and not math.isnan(val_auc) and val_auc > best_auc:
            best_auc = val_auc
            torch.save(model.state_dict(), "best_twotower_model.pt")
            print("Saved new best model based on validation AUC.")

    # Load best model (if saved)
    if os.path.exists("best_twotower_model.pt"):
        model.load_state_dict(torch.load("best_twotower_model.pt", map_location=cfg.device))
        print("Loaded best model from checkpoint.")

    # Final pointwise evaluation on test set (AUC / loss)
    print("\nCreating test DataLoader for pointwise metrics...")
    test_dataset = TwoTowerDataset(test_df_norm, user2idx, item2idx, lang2idx)
    test_loader = DataLoader(
        test_dataset,
        batch_size=cfg.batch_size,
        shuffle=False,
        num_workers=2,
        pin_memory=True,
    )

    test_loss, test_auc = evaluate_pointwise(model, test_loader, cfg.device)
    print(f"Test loss: {test_loss:.4f}, Test AUC: {test_auc:.4f}")

    # Robust ranking evaluation with negative sampling (Recall@K, NDCG@K)
    print("\nBuilding normalized item feature table for ranking evaluation...")
    item_table_norm = build_item_feature_table_norm(train_pool, test_df, means, stds)

    print("Evaluating ranking metrics with negative sampling...")
    rank_metrics = evaluate_ranking_with_neg_sampling(
        model=model,
        test_df=test_df,
        train_pool=train_pool,
        item_table=item_table_norm,
        user2idx=user2idx,
        item2idx=item2idx,
        lang2idx=lang2idx,
        device=cfg.device,
        k_list=cfg.eval_k_list,
        num_negatives=cfg.eval_num_negatives,
    )

    print(f"Ranking evaluation over {rank_metrics['num_users']} users:")
    for k in cfg.eval_k_list:
        print(
            f"  Recall@{k}: {rank_metrics.get(f'Recall@{k}', float('nan')):.4f}, "
            f"NDCG@{k}: {rank_metrics.get(f'NDCG@{k}', float('nan')):.4f}"
        )


if __name__ == "__main__":
    main()

Loading raw data...
Train pool size: 567339, Test size: 141832
Splitting train pool into train and validation by user...
Filtering training set to keep only POSITIVES (target=1)...
Train size (pos only): 293817, Val size (mixed): 113522
Building ID mappings (users/items/langs)...
Num users: 10000, num items: 365628, num langs: 119
[Text] Loaded README embeddings for 9336 / 365628 items (2.55%).
Fitting numeric scalers on training pool...
Normalizing numeric features...
Creating dataloaders for train and validation...
Initializing model...

Epoch 1/10
Train loss (CrossEntropy): 8.3296
Val loss (Pointwise): 0.6902, Val AUC: 0.5015
Saved new best model based on validation AUC.

Epoch 2/10
Train loss (CrossEntropy): 8.2291
Val loss (Pointwise): 0.6783, Val AUC: 0.4907

Epoch 3/10
Train loss (CrossEntropy): 7.5998
Val loss (Pointwise): 0.6790, Val AUC: 0.4950

Epoch 4/10
Train loss (CrossEntropy): 7.0420
Val loss (Pointwise): 0.6903, Val AUC: 0.4847

Epoch 5/10
Train loss (CrossEntropy): 6.